In [1]:
# imports
import os
import pandas as pd

In [2]:
# define base path
base_path = os.getcwd()

# load data
df_airports_geolocation = pd.read_csv(os.path.join (base_path, "data", "raw", "airports_geolocation.csv"))
df_cancelled_diverted = pd.read_csv(os.path.join (base_path, "data", "raw", "Cancelled_Diverted_2023.csv"))
df_flights = pd.read_csv(os.path.join (base_path, "data", "raw", "US_flights_2023.csv"))
df_meteo = pd.read_csv(os.path.join (base_path, "data", "raw", "weather_meteo_by_airport.csv"))

In [3]:
# perform left join on airports geolocation
df_master = df_flights.merge(
    df_airports_geolocation,
    left_on="Dep_Airport",
    right_on="IATA_CODE",
    how="left"
)

In [4]:
# drop reduntant column
df_master.drop(columns=["IATA_CODE"], inplace=True)

In [6]:
# show df
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6743404 entries, 0 to 6743403
Data columns (total 30 columns):
 #   Column              Dtype  
---  ------              -----  
 0   FlightDate          object 
 1   Day_Of_Week         int64  
 2   Airline             object 
 3   Tail_Number         object 
 4   Dep_Airport         object 
 5   Dep_CityName        object 
 6   DepTime_label       object 
 7   Dep_Delay           int64  
 8   Dep_Delay_Tag       int64  
 9   Dep_Delay_Type      object 
 10  Arr_Airport         object 
 11  Arr_CityName        object 
 12  Arr_Delay           int64  
 13  Arr_Delay_Type      object 
 14  Flight_Duration     int64  
 15  Distance_type       object 
 16  Delay_Carrier       int64  
 17  Delay_Weather       int64  
 18  Delay_NAS           int64  
 19  Delay_Security      int64  
 20  Delay_LastAircraft  int64  
 21  Manufacturer        object 
 22  Model               object 
 23  Aicraft_age         int64  
 24  AIRPORT             obje

In [7]:
# convert date column in weather data to datetime format
df_meteo["time"] = pd.to_datetime(df_meteo["time"])
df_master["FlightDate"] = pd.to_datetime(df_master["FlightDate"])

In [8]:
# perform left join meteo data
df_master = df_master.merge(
    df_meteo,
    left_on=["FlightDate", "Dep_Airport"],
    right_on=["time", "airport_id"],
    how="left"
)

In [9]:
# drop redundant columns
df_master.drop(columns=["time", "airport_id"], inplace=True)

In [10]:
# show df
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6743404 entries, 0 to 6743403
Data columns (total 38 columns):
 #   Column              Dtype         
---  ------              -----         
 0   FlightDate          datetime64[ns]
 1   Day_Of_Week         int64         
 2   Airline             object        
 3   Tail_Number         object        
 4   Dep_Airport         object        
 5   Dep_CityName        object        
 6   DepTime_label       object        
 7   Dep_Delay           int64         
 8   Dep_Delay_Tag       int64         
 9   Dep_Delay_Type      object        
 10  Arr_Airport         object        
 11  Arr_CityName        object        
 12  Arr_Delay           int64         
 13  Arr_Delay_Type      object        
 14  Flight_Duration     int64         
 15  Distance_type       object        
 16  Delay_Carrier       int64         
 17  Delay_Weather       int64         
 18  Delay_NAS           int64         
 19  Delay_Security      int64         
 20  De

In [11]:
# convert date column to datetime format
df_cancelled_diverted["FlightDate"] = pd.to_datetime(df_cancelled_diverted["FlightDate"])

# select relevant columns
df_cancelled_diverted = df_cancelled_diverted[["FlightDate", "Tail_Number", "Cancelled", "Diverted"]]

# perform join operation
df_master = df_master.merge(
    df_cancelled_diverted,
    on=["FlightDate", "Tail_Number"],
    how="left"
)

# fill NaN values with 0 if exist
df_master["Cancelled"] = df_master["Cancelled"].fillna(0).astype(int)
df_master["Diverted"] = df_master["Diverted"].fillna(0).astype(int)

In [12]:
# show df
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6825926 entries, 0 to 6825925
Data columns (total 40 columns):
 #   Column              Dtype         
---  ------              -----         
 0   FlightDate          datetime64[ns]
 1   Day_Of_Week         int64         
 2   Airline             object        
 3   Tail_Number         object        
 4   Dep_Airport         object        
 5   Dep_CityName        object        
 6   DepTime_label       object        
 7   Dep_Delay           int64         
 8   Dep_Delay_Tag       int64         
 9   Dep_Delay_Type      object        
 10  Arr_Airport         object        
 11  Arr_CityName        object        
 12  Arr_Delay           int64         
 13  Arr_Delay_Type      object        
 14  Flight_Duration     int64         
 15  Distance_type       object        
 16  Delay_Carrier       int64         
 17  Delay_Weather       int64         
 18  Delay_NAS           int64         
 19  Delay_Security      int64         
 20  De

In [13]:
# save file to output path
output_path = os.path.join (base_path, "data", "processed", "master_file.csv")
df_master.to_csv(output_path, index=False)
print (f"File succesfully saved to {output_path}")

File succesfully saved to c:\Users\emreo\Desktop\ZHAW\PGA\flight_delays\data\processed\master_file.csv
